In [181]:
import sklearn
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt


#todo
#elo by surface type
#elo decay (e.g injury)
#elo fatigue
#regression into new season? FiveThirtyEight NFL (implementation example on Kaggle)
#backtest: kelly betting, kelly betting with filters

In [199]:
#Binds yearly results into one dataframe and outputs as a raw (no cleaning) csv 

folder = 'MensData/'

results_df = pd.read_excel(folder + '2003.xls')

#xls until 2012 inclusive
for i in range (2004,2013):
    toappend_df = pd.read_excel(folder + str(i) + '.xls')
    results_df = pd.concat([results_df,toappend_df], axis=0, ignore_index=True)
    

#for i in range (2013,2019):
#    toappend_df = pd.read_excel(folder + str(i) + '.xlsx')
#    results_df = pd.concat([results_df,toappend_df], axis=0, ignore_index=True)

results_df.to_csv(folder + 'matches_raw.csv')     

In [226]:
#Data cleaning on the raw csv  
#Want format to be 'Surname Initials'  
#All initials followed by'.', only a space between surname and initials, no special characters e.g: '-' 
#Need to add something that replaces all '. ' with ','. Manually done in excel and saves as matches_manual2.csv for now

folder = 'MensData/'
file = 'matches_raw.csv'

results_df = pd.read_csv(folder + file)

def replace_name(name_from, name_to):
    count = 0
    count = count + len(results_df.loc[results_df.Winner == name_from,'Winner'])
    count = count + len(results_df.loc[results_df.Winner == (name_from + ' '),'Winner'])
    count = count + len(results_df.loc[results_df.Loser == name_from,'Loser'])
    count = count + len(results_df.loc[results_df.Loser == (name_from + ' '),'Loser'])
    
    results_df.loc[results_df.Winner == name_from,'Winner'] = name_to
    results_df.loc[results_df.Winner == (name_from + ' '),'Winner'] = name_to
    results_df.loc[results_df.Loser == name_from,'Loser'] = name_to
    results_df.loc[results_df.Loser == (name_from + ' '),'Loser'] = name_to
    
    return(count)

df_corrections = pd.read_csv(folder + 'tennisdata_corrections.csv')
count_corrections = 0

for row in df_corrections.itertuples():
        idx = row.Index
        count_corrections = count_corrections + replace_name(df_corrections.old[idx], df_corrections.new[idx])

#Remove whitespace
new = pd.Series([])
new = results_df['Winner'].str.strip()
results_df['Winner'] = new
new = results_df['Loser'].str.strip()
results_df['Loser'] = new

results_df.W1 = results_df[['W1']].convert_objects(convert_numeric=True).fillna(0)
results_df.W2 = results_df[['W2']].convert_objects(convert_numeric=True).fillna(0)
results_df.W3 = results_df[['W3']].convert_objects(convert_numeric=True).fillna(0)
results_df.W4 = results_df[['W4']].convert_objects(convert_numeric=True).fillna(0)
results_df.W5 = results_df[['W5']].convert_objects(convert_numeric=True).fillna(0)
results_df.L1 = results_df[['L1']].convert_objects(convert_numeric=True).fillna(0)
results_df.L2 = results_df[['L2']].convert_objects(convert_numeric=True).fillna(0)
results_df.L3 = results_df[['L3']].convert_objects(convert_numeric=True).fillna(0)
results_df.L4 = results_df[['L4']].convert_objects(convert_numeric=True).fillna(0)
results_df.L5 = results_df[['L5']].convert_objects(convert_numeric=True).fillna(0)



#Make columns numeric
results_df[['W2','W3', 'L2', 'L3']] = results_df[['W2','W3', 'L2', 'L3']].apply(pd.to_numeric)
        
results_df.to_csv(folder + 'matches_cleaned.csv')

'There were: ' + str(count_corrections) + ' corrections'

/Users/conordavis/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,20,26,29,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/conordavis/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:39: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/conordavis/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:40: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/conordavis/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:41: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/conordavis/anaconda/lib/python3.6/site-packages/ipykernel/__main__.

'There were: 913 corrections'

In [280]:
folder = 'MensData/'
file = 'matches_cleaned.csv'
results_df = pd.read_csv(folder + file)

#Create a datetime index
#format = '%d/%m/%Y'
#results_df.Date = pd.to_datetime(results_df['Date'], format = '%d/%m/%Y')
results_df.Date = pd.to_datetime(results_df['Date'], format = '%Y/%m/%d')
results_df.sort('Date')

#Find best odds
results_df['W_Odds'] = results_df[['B365W','PSW']].max(axis=1)
results_df['L_Odds'] = results_df[['B365L','PSL']].max(axis=1)

#Length of match in games
results_df['Games'] = results_df.W1 + results_df.W2 + results_df.W3 + results_df.W4 + results_df.W5 + results_df.L1 + results_df.L2 + results_df.L3 + results_df.L4 + results_df.L5

#Take a basic slice to test Elo (with surface filter)
basic_df = results_df[['Date', 'Winner', 'Loser', 'Surface', 'W_Odds', 'L_Odds', 'W1', 'W2', 'W3', 'W4', 'W5', 'Games']]
basic_df = basic_df[basic_df.Surface=='Hard']
basic_df['New_Index'] = range(0, len(basic_df))
basic_df = basic_df.set_index('New_Index')

#Encoding names: losers first as there may be players with no wins
le = LabelEncoder()
basic_df.Loser = le.fit_transform(basic_df.Loser)
basic_df.Winner = le.transform(basic_df.Winner)

#Use this to list all players: list(le.inverse_transform(range(1,n_players)))
basic_df

/Users/conordavis/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/conordavis/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Date,Winner,Loser,Surface,W_Odds,L_Odds,W1,W2,W3,W4,W5,Games
New_Index,,,,,,,,,,,,
0,2002-12-30,132,355,Hard,NaN,NaN,6.0,7.0,0.0,0.0,0.0,21.0
1,2002-12-30,314,552,Hard,NaN,NaN,2.0,6.0,6.0,0.0,0.0,26.0
2,2002-12-30,589,515,Hard,1.364,2.875,3.0,6.0,6.0,0.0,0.0,27.0
3,2002-12-30,629,310,Hard,NaN,NaN,4.0,6.0,6.0,0.0,0.0,24.0
4,2002-12-30,654,527,Hard,NaN,NaN,6.0,4.0,7.0,0.0,0.0,30.0
5,2002-12-30,687,668,Hard,NaN,NaN,6.0,6.0,0.0,0.0,0.0,17.0
6,2002-12-31,166,43,Hard,1.667,2.100,6.0,6.0,0.0,0.0,0.0,19.0
7,2002-12-31,374,440,Hard,1.400,2.750,6.0,6.0,0.0,0.0,0.0,19.0
8,2002-12-31,405,424,Hard,1.667,2.100,6.0,6.0,0.0,0.0,0.0,16.0


In [281]:
#Player Fatigue

def player_fatigue(df, player, date, window, discount):
    fatigue = 0.
    date = np.datetime64(date)
    
    df_filter = df[df.Winner == player]
    df_filter = df_filter[df_filter.Date < date]
    df_filter = df_filter[df_filter.Date > (date - np.timedelta64(window + 1,'D'))]
    dates = df_filter.Date.values
    
    
    if(len(dates) > 0):
        days = (date - dates)/86400000000000
        games = df_filter.Games.values
        fatigues = np.ones(len(games))
        for i in range(0, len(games)):
            fatigues[i] = discount**(int(days[i])) * float(games[i])
            fatigue = fatigue + fatigues[i]
    return(fatigue)
    

basic_df['w_fatigue'] = 0.
basic_df['l_fatigue'] = 0.
for row in basic_df.itertuples():
    basic_df.at[row.Index, 'w_fatigue'] = player_fatigue(basic_df, row.Winner, row.Date, 3, 0.75)
    basic_df.at[row.Index, 'l_fatigue'] = player_fatigue(basic_df, row.Loser, row.Date, 3, 0.75)
    
basic_df

,Date,Winner,Loser,Surface,W_Odds,L_Odds,W1,W2,W3,W4,W5,Games,w_fatigue,l_fatigue
New_Index,,,,,,,,,,,,,,
0,2002-12-30,132,355,Hard,NaN,NaN,6.0,7.0,0.0,0.0,0.0,21.0,0.000000,0.000000
1,2002-12-30,314,552,Hard,NaN,NaN,2.0,6.0,6.0,0.0,0.0,26.0,0.000000,0.000000
2,2002-12-30,589,515,Hard,1.364,2.875,3.0,6.0,6.0,0.0,0.0,27.0,0.000000,0.000000
3,2002-12-30,629,310,Hard,NaN,NaN,4.0,6.0,6.0,0.0,0.0,24.0,0.000000,0.000000
4,2002-12-30,654,527,Hard,NaN,NaN,6.0,4.0,7.0,0.0,0.0,30.0,0.000000,0.000000
5,2002-12-30,687,668,Hard,NaN,NaN,6.0,6.0,0.0,0.0,0.0,17.0,0.000000,0.000000
6,2002-12-31,166,43,Hard,1.667,2.100,6.0,6.0,0.0,0.0,0.0,19.0,0.000000,0.000000
7,2002-12-31,374,440,Hard,1.400,2.750,6.0,6.0,0.0,0.0,0.0,19.0,0.000000,0.000000
8,2002-12-31,405,424,Hard,1.667,2.100,6.0,6.0,0.0,0.0,0.0,16.0,0.000000,0.000000


In [303]:
def update_elo(winner_elo, loser_elo, elo_mean, elo_width, k_factor):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expected_win = expected_result(winner_elo, loser_elo, elo_width)
    change_in_elo = k_factor * (1-expected_win)
    winner_elo += change_in_elo
    loser_elo -= change_in_elo
    return winner_elo, loser_elo

def expected_result(elo_a, elo_b, elo_width):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a
                              
def update_end_of_season(elos):
    """Regression towards the mean
    
    Following 538 nfl methods
    https://fivethirtyeight.com/datalab/nfl-elo-ratings-are-back/
    """
    diff_from_mean = elos - elo_mean
    elos -= diff_from_mean/3
    return elos

def player_stats(matches, initial_values, n_players):
        stats = pd.DataFrame(index=matches.Date.unique(), columns=range(n_players))
        stats.sort_index()
        stats.iloc[0, :] = initial_values
        return(stats)
    
def simulation_elo(elo_mean, elo_width, k_factor, f_factor):

    basic_df['w_elo_before_game'] = 0
    basic_df['w_elo_after_game'] = 0
    basic_df['w_games'] = 0
    basic_df['l_elo_before_game'] = 0
    basic_df['l_elo_after_game'] = 0
    basic_df['l_games'] = 0
    
    n_players = len(le.classes_)
    loss_log = 0
    loss_logb = 0
    loss_count = 0
    
    #Dataframe to hold player elos as function of time
    current_elos = np.ones(shape=(n_players)) * elo_mean
    df_player_elos = player_stats(basic_df, current_elos, n_players)

    #Dataframe to hold player games as a function of time
    current_games = np.zeros(shape=(n_players))
    df_player_games = player_stats(basic_df, current_games, n_players)

    #Array to hold compounded edge and PNL per day
    edge_compounded = np.zeros(shape=len(basic_df))
    pnl = np.zeros(shape=len(basic_df))

    #current_season = df_concat.at[0,Season']
    for row in basic_df.itertuples():

        #abbreviations
        idx = row.Index
        today = row.Date
        w_id = row.Winner
        l_id = row.Loser
        w_odds = row.W_Odds
        l_odds = row.L_Odds
        w_fatigue = row.w_fatigue
        l_fatigue = row.l_fatigue

        # Get current elos, games and fatigue
        w_elo_before = current_elos[w_id]
        l_elo_before = current_elos[l_id]
        w_games = current_games[w_id]
        l_games = current_games[l_id]

        # Update on game results
        w_elo_after, l_elo_after = update_elo(w_elo_before, l_elo_before, elo_mean, elo_width, k_factor)

        # Save updated elos
        basic_df.at[idx, 'w_elo_before_game'] = w_elo_before
        basic_df.at[idx, 'l_elo_before_game'] = l_elo_before
        basic_df.at[idx, 'w_elo_after_game'] = w_elo_after
        basic_df.at[idx, 'l_elo_after_game'] = l_elo_after
        current_elos[w_id] = w_elo_after
        current_elos[l_id] = l_elo_after

        # Save elos to DataFrame
        df_player_elos.at[today, w_id] = w_elo_after
        df_player_elos.at[today, l_id] = l_elo_after

        # Update games played
        basic_df.at[idx, 'w_games'] = w_games
        basic_df.at[idx, 'l_games'] = l_games
        df_player_games.at[today, w_id] = w_games + 1
        df_player_games.at[today, l_id] = l_games + 1
        current_games[w_id] = current_games[w_id] + 1
        current_games[l_id] = current_games[l_id] + 1
        
        if ((w_games > 20) & (l_games > 20)) & (w_odds > 1) & (l_odds > 1):
            adjustment = f_factor * (w_fatigue - l_fatigue)
            loss_log = loss_log + np.log(expected_result(w_elo_before - adjustment, l_elo_before, elo_width))
            loss_logb = loss_logb + np.log((1/w_odds + 1 - 1/l_odds)/2)
            loss_count = loss_count + 1

    df_player_elos.fillna(method='ffill', inplace = 'True')
    df_player_games.fillna(method='ffill', inplace = 'True')
    
    #return(loss_log/loss_count, loss_logb/loss_count)
    
    


simulation_elo(1500, 400, 25, 1.2)



In [304]:
basic_df

,Date,Winner,Loser,Surface,W_Odds,L_Odds,W1,W2,W3,W4,W5,Games,w_fatigue,l_fatigue,w_elo_before_game,w_elo_after_game,w_games,l_elo_before_game,l_elo_after_game,l_games
New_Index,,,,,,,,,,,,,,,,,,,,
0,2002-12-30,132,355,Hard,NaN,NaN,6.0,7.0,0.0,0.0,0.0,21.0,0.000000,0.000000,1500,1512,0,1500,1487,0
1,2002-12-30,314,552,Hard,NaN,NaN,2.0,6.0,6.0,0.0,0.0,26.0,0.000000,0.000000,1500,1512,0,1500,1487,0
2,2002-12-30,589,515,Hard,1.364,2.875,3.0,6.0,6.0,0.0,0.0,27.0,0.000000,0.000000,1500,1512,0,1500,1487,0
3,2002-12-30,629,310,Hard,NaN,NaN,4.0,6.0,6.0,0.0,0.0,24.0,0.000000,0.000000,1500,1512,0,1500,1487,0
4,2002-12-30,654,527,Hard,NaN,NaN,6.0,4.0,7.0,0.0,0.0,30.0,0.000000,0.000000,1500,1512,0,1500,1487,0
5,2002-12-30,687,668,Hard,NaN,NaN,6.0,6.0,0.0,0.0,0.0,17.0,0.000000,0.000000,1500,1512,0,1500,1487,0
6,2002-12-31,166,43,Hard,1.667,2.100,6.0,6.0,0.0,0.0,0.0,19.0,0.000000,0.000000,1500,1512,0,1500,1487,0
7,2002-12-31,374,440,Hard,1.400,2.750,6.0,6.0,0.0,0.0,0.0,19.0,0.000000,0.000000,1500,1512,0,1500,1487,0
8,2002-12-31,405,424,Hard,1.667,2.100,6.0,6.0,0.0,0.0,0.0,16.0,0.000000,0.000000,1500,1512,0,1500,1487,0


In [320]:
def expected_result(elo_a, elo_b, elo_width):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a

#Betting parameters
bankroll = 100.0
bankroll_noedge = bankroll
kelly_fraction = 1
spread_cost = 0.035
wins = 0
loses = 0
arbs = 0
nobets = 0

#Array to hold compounded edge and PNL per day
edge_compounded = np.zeros(shape=len(basic_df))
pnl = np.zeros(shape=len(basic_df))

for row in basic_df.itertuples():
    idx = row.Index
    today = row.Date
    w_id = row.Winner
    l_id = row.Loser
    w_odds = row.W_Odds
    l_odds = row.L_Odds
    w_elo_before = row.w_elo_before_game
    l_elo_before = row.l_elo_before_game
    w_games = row.w_games
    l_games = row.l_games
    
    #Betting logic
    if ((w_odds > 1.25) & (l_odds > 1.25) & (w_games > 20) & (l_games > 20)):
        elo_l_odds = 1/expected_result(l_elo_before, w_elo_before, 400)
        elo_w_odds = 1/expected_result(w_elo_before, l_elo_before, 400)
        if (elo_l_odds < l_odds) & (elo_w_odds < w_odds):
            arbs = arbs + 1
        elif elo_l_odds < l_odds:
            edge = l_odds/elo_l_odds - 1
            if (edge > 0.15):
                perc_stake = kelly_fraction*edge/(l_odds - 1)
                abs_stake = perc_stake*bankroll
                bankroll = bankroll - abs_stake
                bankroll_noedge = (1-perc_stake*spread_cost)*bankroll_noedge
                pnl[idx] = pnl[idx] + perc_stake
                edge_compounded[idx] = (1+edge_compounded[idx])*(1+edge)-1
                loses = loses + 1
        elif elo_w_odds < w_odds:
            edge = w_odds/elo_w_odds - 1
            if (edge > 0.15):
                perc_stake = kelly_fraction*edge/(w_odds - 1)
                abs_stake = perc_stake*bankroll
                bankroll = bankroll + abs_stake*(w_odds-1)
                bankroll_noedge = (1-perc_stake*spread_cost)*bankroll_noedge
                pnl[idx] = pnl[idx] + perc_stake*(w_odds-1)
                edge_compounded[idx] = (1+edge_compounded[idx])*(1+edge)-1
                wins = wins + 1
        
pnl

array([0.       , 0.       , 0.       , ..., 0.       , 0.4103288,
       0.       ])

In [321]:
bankroll

3.022610040639159e-51

In [40]:
players_tograph = ['Federer R.', 'Djokovic N.', 'Nadal R.', 'Murray A.', 'Zverev A.']
players_transformed = le.transform(players_tograph)

plt.plot(df_player_elos[players_transformed[0]]) # blue
plt.plot(df_player_elos[players_transformed[1]]) # orange
plt.plot(df_player_elos[players_transformed[2]]) # green
plt.plot(df_player_elos[players_transformed[3]]) # red
plt.plot(df_player_elos[players_transformed[4]]) # purple
plt.show()


NameError: name 'df_player_elos' is not defined

In [ ]:

'''
#Basic Elo Calculation

#Elo parameters
elo_mean = 1500
elo_width = 400
k_factor = 32

#Betting parameters
bankroll = 100.0
bankroll_noedge = bankroll
kelly_fraction = 1
spread_cost = 0.035
wins = 0
loses = 0
arbs = 0
nobets = 0

basic_df['w_elo_before_game'] = 0
basic_df['w_elo_after_game'] = 0
basic_df['w_games'] = 0
basic_df['l_elo_before_game'] = 0
basic_df['l_elo_after_game'] = 0
basic_df['l_games'] = 0
n_players = len(le.classes_)

#Dataframe to hold player elos as function of time
current_elos = np.ones(shape=(n_players)) * elo_mean
df_player_elos = player_stats(basic_df, current_elos, n_players)

#Dataframe to hold player games as a function of time
current_games = np.zeros(shape=(n_players))
df_player_games = player_stats(basic_df, current_games, n_players)

#Array to hold compounded edge and PNL per day
edge_compounded = np.zeros(shape=len(basic_df))
pnl = np.zeros(shape=len(basic_df))

#current_season = df_concat.at[0,Season']
for row in basic_df.itertuples():
    
    #abbreviations
    idx = row.Index
    today = row.Date
    w_id = row.Winner
    l_id = row.Loser
    w_odds = row.W_Odds
    l_odds = row.L_Odds
    
    # Get current elos and games
    w_elo_before = current_elos[w_id]
    l_elo_before = current_elos[l_id]
    w_games = current_games[w_id]
    l_games = current_games[l_id]
    
    # Update on game results
    w_elo_after, l_elo_after = update_elo(w_elo_before, l_elo_before, elo_mean, elo_width, k_factor)
        
    # Save updated elos
    basic_df.at[idx, 'w_elo_before_game'] = w_elo_before
    basic_df.at[idx, 'l_elo_before_game'] = l_elo_before
    basic_df.at[idx, 'w_elo_after_game'] = w_elo_after
    basic_df.at[idx, 'l_elo_after_game'] = l_elo_after
    current_elos[w_id] = w_elo_after
    current_elos[l_id] = l_elo_after
    
    # Save elos to DataFrame
    df_player_elos.at[today, w_id] = w_elo_after
    df_player_elos.at[today, l_id] = l_elo_after
    
    # Update games played
    df_player_games.at[today, w_id] = w_games + 1
    df_player_games.at[today, l_id] = l_games + 1
    current_games[w_id] = current_games[w_id] + 1
    current_games[l_id] = current_games[l_id] + 1
    
    #Betting logic
    if ((w_odds > 1) & (l_odds > 1) & (w_games > 150) & (l_games > 150)):
        elo_l_odds = 1/expected_result(l_elo_before, w_elo_before)
        elo_w_odds = 1/expected_result(w_elo_before, l_elo_before)
        if (elo_l_odds < l_odds) & (elo_w_odds < w_odds):
            arbs = arbs + 1
        elif elo_l_odds < l_odds:
            edge = l_odds/elo_l_odds - 1
            if edge < 0.1:
                perc_stake = kelly_fraction*edge/(l_odds - 1)
                abs_stake = perc_stake*bankroll
                bankroll = bankroll - abs_stake
                bankroll_noedge = (1-perc_stake*spread_cost)*bankroll_noedge
                pnl[idx] = pnl[idx] - abs_stake
                edge_compounded[idx] = (1+edge_compounded[idx])*(1+edge)-1
                loses = loses + 1
        elif elo_w_odds < w_odds:
            edge = w_odds/elo_w_odds - 1
            if edge <0.1:
                perc_stake = kelly_fraction*edge/(w_odds - 1)
                abs_stake = perc_stake*bankroll
                bankroll = bankroll + abs_stake*(w_odds-1)
                bankroll_noedge = (1-perc_stake*spread_cost)*bankroll_noedge
                pnl[idx] = pnl[idx] + abs_stake*(w_odds-1)
                edge_compounded[idx] = (1+edge_compounded[idx])*(1+edge)-1
                wins = wins + 1
        
        
df_player_elos.fillna(method='ffill', inplace = 'True')
df_player_games.fillna(method='ffill', inplace = 'True')

'''
